# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# APK Files to load
#file_paths = (
#     "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.0.63-2076_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.5.72-2077_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.0.66-2078_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.7.78-2085_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_11.0.0.72-2086_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#    "../../../Downloads/com.snapchat.android_11.0.1.72-2089_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#    "../../../Downloads/com.snapchat.android_11.0.6.82-2095_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
#)


file_paths = (
# #     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726452_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726384_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168935_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168922_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
    
     "../../../Downloads/com.instagram.android_159.0.0.29.122-244390467_minAPI23(arm64-v8a)(560,640dpi)_apkmirror.com.apk",
     "../../../Downloads/com.instagram.android_159.0.0.29.122-244390338_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_159.0.0.29.122-244390467_minAPI23(arm64-v8a)(560,640dpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_159.0.0.29.122-244390338_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `named_c_decs` in form of a tuple of Strings
* `named_m_decs` in form of a tuple of MethodDeclarations
* `named_f_decs` in form of a tuple of FieldDeclarations

In [6]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.2wk',
    'COMMENT_LONGCLICK_HANDLER': 'X.3To',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.8U5',
    'DM_HOLDER': 'X.1qJ',
    'DM_REPLY': 'X.8Ib',
    'DM_VOICEMESSAGE': 'X.8ZO',
    'DMVIDEOPLAYER': 'X.2y2',
    'FEEDITEM': 'X.3AC',
    'FRAGMENT': 'X.Bk6',
    'HEADERGENERATOR': 'X.0S0',
    'HTTPREQUESTTASKFACTORY': 'X.1ti',
    'INAPP_NOTIFICATION': 'X.8yK',
    'IGTV_VIEWER_FRAGMENT': 'X.3qT',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.CxS',
    'JACKSON_CREATOR': 'X.Cwz',
    'LIVE_POST': 'X.2Or',
    'MAINFEED': 'X.02c',
    'MEDIAITEM': 'X.2mu',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.8yq',
    'NOTIFICATION_HANDLER': 'X.2DT',
    'PHOTOTIMERCONTROLLER': 'X.1DJ',
    'POSTDETAIL': 'X.33y',
    'POSTGESTURE_SIMPLEPOST': 'X.3Me',
    'POSTGESTURE_CAROUSELIMAGE': 'X.3LL',
    'POSTGESTURE_CAROUSELVIDEO': 'X.3Mb',
    'POSTVIEW': 'X.3A9',
    'PROFILE': 'X.0w1',
    'PROFILEDETAIL': 'X.3oV',
    'PROFILEPIC_LONGCLICK': 'X.3oY',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.ELB',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.CRh',
    'NOTIFICATION_RUNNABLE': 'X.8yL',
    'SERIALSCHEDULER': 'X.7mK',
    'SESSION': 'X.0Ly',
    'SIMPLE_EXECUTOR': 'X.0lZ',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.FEi',
    'VIDEOSETTINGSMANAGER': 'X.2Sr',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [7]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.2wk', '<init>', 'X.2Cw', 'X.2x8', 'X.0Ly', 'android.view.View', 'X.2wS', 'X.2wO'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3To', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.Bk7', 'androidx.fragment.app.FragmentActivity', 'X.0Ly', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Ly'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Ly'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.8U5', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.1qJ', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.8Ib', 'A0K', 'X.8Ib', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.8Ib', 'A0H', 'X.8Ib'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.2y2', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.3AC', 'A00', 'X.Cxk'),
    'FRAGMENT_GETACTIVITY': MethodDec('X.Bk6', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0S0', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.1ti', 'A02', 'java.lang.String', 'X.0Ly'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.8yK', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.3qT', 'A00', 'X.3qT'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.3qT', 'A0C', 'X.3qT'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.CxS', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.Cwz', 'A03', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.Cwz', '<init>', 'X.CxF'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.2Or', 'A00', 'X.CxI', 'X.19Q'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'AKk'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.02c', 'configureActionBar', 'X.40J'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.2mu', 'A01', 'X.Cxk', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.CxI', 'X.2mu'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.8yq', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2DT', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.1DJ', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.33y', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.3Me', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.3LL', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.3Mb', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.3A9', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.3oV', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0w1', 'A04', 'X.0Ly', 'X.0w2'),
    'PROFILE_ONLONGCLICK': MethodDec('X.3oY', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.ELB', 'A0T', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1B', 'X.18l'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0j', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0P', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0s', 'X.0pp', 'X.18l'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.CRh', 'A00', 'X.CMN', 'X.CLD'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.8yL', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.7mK', 'A02', 'X.7mG'),
    'SESSION_GETTOKEN': MethodDec('X.0Ly', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.0lZ', 'onFail', 'X.2in'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.0lZ', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.0lZ', '<init>', 'X.0Ly'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.FEi', 'A03', 'X.CxI', 'X.FEj'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.2Sr', 'A00', 'X.0Ly'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [8]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.4dl', 'A07'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.2Cw', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3To', 'A04'),
    'COMMENT_TEXT': FieldDec('X.2qS', 'A0Y'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.8U5', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.1q9', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.1qE', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.8ZO', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.3AC', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.7eu', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.3qT', 'A0E'),
    'MAINFEED_SESSION': FieldDec('X.02c', 'A0T'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.39o', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.18l', 'A09'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.18l', 'A08'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.CRs', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.CMA', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.8yK', 'A00'), FieldDec('X.8yL', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.8yK', 'A01'), FieldDec('X.8yL', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.340', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.3As', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.3od', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.3od', 'A00'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.3od', 'A01'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.3od', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.3oa', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.CRg', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.ELC', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1L'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.7Un', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.18l', 'A0D')
}

In [9]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [10]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [11]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [12]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [13]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	X.02c#configureActionBar(X.40J) -> X.02c#configureActionBar(X.4Al)
+ Found single candidate for Method. Considering it a match 
	X.8yq#A01() -> X.8cS#A01()
+ Found single candidate for Method. Considering it a match 
	X.0S0#A00(android.content.Context) -> X.0SA#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0s(X.0pp, X.18l) -> com.instagram.reels.fragment.ReelViewerFragment#A0s(X.0qh, X.18n)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback)
+ Found single candidate for Method

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [14]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

.. Considering ambiguous match by UniqueString as match with a majority of 0.89 (LX/3qT; -> LX/3ye;)
.. Considering ambiguous match by UniqueString as match with a majority of 0.94 (Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;)
+ Found single candidate for Class. Considering it a match! 
	LX/0Ly; -> LX/0Ly;
+ Found single candidate for Class. Considering it a match! 
	LX/18l; -> LX/18n;
+ Found single candidate for Class. Considering it a match! 
	LX/1q9; -> LX/25u;
+ Found single candidate for Class. Considering it a match! 
	LX/2Cw; -> LX/1DH;
+ Found single candidate for Class. Considering it a match! 
	LX/2DT; -> LX/3a8;
+ Found single candidate for Class. Considering it a match! 
	LX/2Or; -> LX/2M3;
+ Found single candidate for Class. Considering it a match! 
	LX/2mu; -> LX/2N4;
+ Found single candidate for Class. Considering it a match! 
	LX/2qS; -> LX/2VK;
+ Found single candidate for Class. Considering it a match! 
	LX/33y; -> LX/2Wf;
+

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [15]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/2wk; -> LX/2BK;
+ Found single candidate for Class. Considering it a match! 
	LX/3To; -> LX/2hs;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/8U5; -> LX/8D4;
+ Found single candidate for Class. Considering it a match! 
	LX/1qJ; -> LX/264;
+ Found single candidate for Class. Considering it a match! 
	LX/2y2; -> LX/2Cp;
+ Found single candidate for Class. Considering it a match! 
	LX/1ti; -> LX/1aq;
+ Found single candidate for Class. Considering it a match! 
	LX/Cwz; -> LX/CXH;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/1DJ; -> LX/0xh;
* Found 

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [16]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.3Me#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.3LL#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.3Mb#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	X.2wk#<init>(X.2Cw, X.2x8, X.0Ly, android.view.View, X.2wS, X.2wO) -> X.2BK#<init>(X.1DH, X.2Bq, X.0Ly, android.view.View, X.2B3, X.2BC)
+ Found single candidate for Method. Considering it a match 
	X.3To#onLongPress(android.view.MotionEvent) -> X.2hs#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.Bk7, androidx.fragment.app.FragmentActivity, X.0Ly, android.os

+ Found single candidate for Method. Considering it a match 
	X.0Ly#getToken() -> X.0Ly#getToken()
+ Found single candidate for Method. Considering it a match 
	X.FEi#A03(X.CxI, X.FEj) -> X.FGR#A03(X.CWt, X.FGS)
Could resolve 0 new Classes, 5 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [17]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/19Q; -> LX/1A6;
+ Found single candidate for Class. Considering it a match! 
	LX/3AD; -> LX/2oW;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/FEj; -> LX/FGS;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/1PV; -> LX/1D8;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a match!

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [18]:
print(len(accumulator.matching_cs), "/", len(named_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

72 / 46 Classes were matched
55 / 60 Methods were matched
37 / 39 Fields were matched

Matching Classes:
• X.02c -> X.02c
• X.8yq -> X.8cS
• X.0S0 -> X.0SA
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.8Ib -> X.82D
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.0lZ -> X.0lh
• X.2Sr -> X.2y3
• X.CRh -> X.CJc
• X.3oV -> X.3T2
• X.8yK -> X.8eZ
• X.8yL -> X.8eb
• X.ELB -> X.EAQ
• X.0Ly -> X.0Ly
• X.18l -> X.18n
• X.1q9 -> X.25u
• X.2Cw -> X.1DH
• X.2DT -> X.3a8
• X.2Or -> X.2M3
• X.2mu -> X.2N4
• X.2qS -> X.2VK
• X.33y -> X.2Wf
• X.3AC -> X.2oU
• X.3qT -> X.3ye
• X.4dl -> X.78B
• X.7eu -> X.8oa
• X.7mK -> X.795
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• com.in

• com.instagram.reels.fragment.ReelViewerFragment#A1B(X.18l) -> com.instagram.reels.fragment.ReelViewerFragment#A1B(X.18n)
• com.instagram.reels.fragment.ReelViewerFragment#A0j(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String) -> com.instagram.reels.fragment.ReelViewerFragment#A0j(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String)
• com.instagram.reels.fragment.ReelViewerFragment#A0P(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0P(com.instagram.reels.fragment.ReelViewerFragment)
• X.7mK#A02(X.7mG) -> X.795#A02(X.79D)
• X.0lZ#<init>(X.0Ly) -> X.0lh#<init>(X.0Ly)
• com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent) -> com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent)
• com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int) -> com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int)
• X.8U5#onLongPre

In [19]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(named_c_decs.values(), r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.2BK', '<init>', 'X.1DH', 'X.2Bq', 'X.0Ly', 'android.view.View', 'X.2B3', 'X.2BC'),
    MethodDec('X.2hs', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.Bdp', 'androidx.fragment.app.FragmentActivity', 'X.0Ly', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ly),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ly),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerCont

In [20]:
"""
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"
"""


c_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"


In [21]:
%autoreload

c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ly)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ly)
No matching Method found for X.3Me#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.3LL#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.3Mb#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.340#A06
No matching Field found for X.3As#A02


In [22]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.2BK")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.2hs")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.8D4")

    override val DM_HOLDER = ClassDec("X.264")

    override val DM_REPLY = ClassDec("X.82D")

    override val DM_VOICEMESSAGE = ClassDec("X.83u")

    override val DMVIDEOPLAYER = ClassDec("X.2Cp")

    override val FEEDITEM = ClassDec("X.2oU")

    override val FRAGMENT = ClassDec("X.Bdo")

    override v

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.78B"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A07")

    @FieldClass(["X.1DH"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.2hs"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.2VK"])
    override val COMMENT_TEXT = VariableDec<String>("A0Y")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    @FieldClass(

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```